In [3]:
#This is a streamlined call that only fetches data for all Batteries mentioned in a list - actual generation in MW h, aggregated across all BM Units but with discharge and charge info separately

import requests
import pandas as pd
import os
from datetime import date

# Set your desired date range
start_date = date(2024, 6, 30)
end_date = date(2025, 7, 1)

# Load BMU to fuel type mapping
mapping_path = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\BESS_Master_List.csv"
mapping_df = pd.read_csv(mapping_path)

# No filtering – include all BM Units
bm_units = mapping_df['NESO BMU ID'].dropna().unique().tolist()

if not bm_units:
    raise ValueError("No BM Units found in the mapping data.")

# API request setup
base_url = "https://data.elexon.co.uk/bmrs/api/v1/datasets/B1610/stream"
params = {
    "from": start_date.isoformat(),
    "to": end_date.isoformat(),
    "bmUnit": bm_units
}

# Fetch B1610 generation data for filtered BM Units only
response = requests.get(base_url, params=params)
response.raise_for_status()
data = response.json()

# Convert to DataFrame
df = pd.DataFrame(data)
if df.empty:
    raise ValueError("No data returned from B1610 API.")

# Merge API data with filtered fuel type mapping
df_merged = df.merge(mapping_df, left_on="nationalGridBmUnitId", right_on="NESO BMU ID", how="left")

# Split into positive and negative generation
df_merged['positive_generation'] = df_merged['quantity'].apply(lambda x: x if x > 0 else 0)
df_merged['negative_generation'] = df_merged['quantity'].apply(lambda x: x if x < 0 else 0)

# Aggregate by fuel type, date, and period
agg_df = df_merged.groupby(
    ['settlementDate', 'settlementPeriod', 'REG FUEL TYPE']
).agg({
    'positive_generation': 'sum',
    'negative_generation': 'sum'
}).reset_index()

# Pivot
pivot_pos = agg_df.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='REG FUEL TYPE',
    values='positive_generation'
).add_prefix('pos_')

pivot_neg = agg_df.pivot_table(
    index=['settlementDate', 'settlementPeriod'],
    columns='REG FUEL TYPE',
    values='negative_generation'
).add_prefix('neg_')

# Combine
pivot_df = pd.concat([pivot_pos, pivot_neg], axis=1).reset_index()

# Ensure all expected columns are present
expected_cols = ['pos_BATTERY', 'neg_BATTERY']
for col in expected_cols:
    if col not in pivot_df.columns:
        pivot_df[col] = 0

pivot_df = pivot_df[['settlementDate', 'settlementPeriod'] + expected_cols]

# --- Save to CSV ---
start_str = start_date.strftime("%Y-%m-%d")
end_str = end_date.strftime("%Y-%m-%d")
filename = f"BMunitAgg_BATTERY{start_str}_to_{end_str}.csv"

output_dir = r"C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, filename)

pivot_df.to_csv(output_file, index=False)
print(f"Saved to {output_file}")


Saved to C:\Users\spice\Dropbox\Documents\Imperial 2024.2025\MECH70038 - Research Projects\_My Thesis\Data\BMunitAgg_BATTERY2024-06-30_to_2025-07-01.csv
